In [150]:
import nltk
import os
import pandas as pd
import numpy as np
import json
import re

In [151]:
MOVIE_DATA = 'movies_metadata.csv'

In [155]:
with open(MOVIE_DATA, 'r') as f:
    movie_data_df = pd.read_csv(f)
    movie_titles = sorted(list(filter(lambda x: not pd.isnull(x), movie_data_df['original_title'])))

In [156]:
STEAM_DATA = os.path.join('steam_data', 'steam_data.csv')
AVAILABLE_GAMES = os.path.join('steamData', '80k_data', 'available_games.json')

In [157]:
available_game_ids = json.load(open(AVAILABLE_GAMES, 'r'))

In [158]:
with open(STEAM_DATA, 'r') as f:
    steam_data_df = pd.read_csv(f)
    fn = lambda x: x.url.split('/')[4]
    steam_data_df['app_id'] = steam_data_df.apply(fn, axis=1)

In [159]:
all_ids = set(steam_data_df['app_id'])
available_game_ids_set = set(map(str, available_game_ids))
steam_data_df_filtered = steam_data_df[steam_data_df['app_id'].isin(available_game_ids_set)]

In [163]:
steam_titles = list(sorted(set(steam_data_df_filtered['name'])))

In [190]:
movie_titles_set = set(map(lambda x: x.lower(), movie_titles))
steam_titles_set = set(map(lambda x: x.lower(), steam_titles))

In [191]:
same_title = {"same_titles": list(movie_titles_set.intersection(steam_titles_set))}
json.dump(same_title, open('./steamData/80k_data/movie_steam_same_titles.json', 'w+'), indent=4)

In [240]:
from difflib import SequenceMatcher
import time


movie_titles_list = sorted(list(movie_titles_set))
steam_titles_list = sorted(list(steam_titles_set))
title_sim_mat = np.zeros((len(movie_titles_list), len(steam_titles_list)))

start = time.time()
for i in range(len(movie_titles_list)):
    for j in range(len(steam_titles_list)):
        title_sim_mat[i,j] = SequenceMatcher(movie_titles_list[i], steam_titles_list[j]).ratio()
    cur_time = time.time()
    if (i % 500 == 0):
        print(f'i = {i}')
        print(f'passed: {cur_time - start}')

i = 0
passed: 0.04380011558532715
i = 500
passed: 18.89723801612854
i = 1000
passed: 35.95387005805969
i = 1500
passed: 52.92241096496582
i = 2000
passed: 71.89214515686035
i = 2500
passed: 90.56083917617798
i = 3000
passed: 108.76062202453613
i = 3500
passed: 124.44826698303223
i = 4000
passed: 139.76490020751953
i = 4500
passed: 155.06018805503845
i = 5000
passed: 170.28989100456238
i = 5500
passed: 185.61902928352356
i = 6000
passed: 200.8940601348877
i = 6500
passed: 216.28936004638672
i = 7000
passed: 232.8297131061554
i = 7500
passed: 248.0979402065277
i = 8000
passed: 263.35445499420166
i = 8500
passed: 278.60399103164673
i = 9000
passed: 294.1783640384674
i = 9500
passed: 309.50119495391846
i = 10000
passed: 324.63406014442444
i = 10500
passed: 339.9628572463989
i = 11000
passed: 355.2036051750183
i = 11500
passed: 370.4774661064148
i = 12000
passed: 385.65658712387085
i = 12500
passed: 401.0644452571869
i = 13000
passed: 416.4612011909485
i = 13500
passed: 432.2381501197815
i 

In [248]:
sim_df = pd.DataFrame(title_sim_mat, columns = steam_titles_list, index = movie_titles_list)

In [254]:
sim_df.to_csv('movie_game_title_similarity.csv')

## Draft

In [214]:
np.argsort(title_sim_mat[0])[::-1]

array([1992, 1344, 2164, ..., 1817, 1816,    0])

In [208]:
print(steam_titles_list[1992], title_sim_mat[0][1992])
print(steam_titles_list[1344], title_sim_mat[0][1344])
print(steam_titles_list[2164], title_sim_mat[0][2164])

sniper art of victory 0.5669467095138409
martial arts brutality 0.3919309008348103
sword art online: fatal bullet 0.3779644730092272


In [215]:
movie_titles_list.index('max payne')

20801

In [233]:
from difflib import SequenceMatcher

for j in range(len(steam_titles_list)):
    title_sim_mat[20801,j] = SequenceMatcher(None, movie_titles_list[20801], steam_titles_list[j]).ratio()

In [235]:
print(movie_titles_list[20801])
list(np.argsort(title_sim_mat[20801])[::-1])[:10]

max payne


[1355, 1357, 1354, 543, 1339, 886, 1699, 113, 616, 1599]

max payne 3 0.9
